In [21]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
from __future__ import annotations

import logging
from time import time
import requests
import uuid
from google.protobuf.json_format import MessageToDict
from typing import Callable, Optional, Union
import random
import vega_sim.grpc.client as vac
import vega_sim.proto.data_node.api.v1 as data_node_protos
import vega_sim.proto.vega as vega_protos
import vega_sim.api.faucet as faucet
from vega_sim.api.helpers import get_enum, enum_to_str, wait_for_acceptance

import vega_sim.proto.vega.api.v1.core_pb2 as core_proto
import vega_sim.proto.vega.commands.v1.commands_pb2 as commands_proto
import vega_sim.proto.vega.commands.v1.transaction_pb2 as transaction_proto
import vega_sim.proto.vega.commands.v1.signature_pb2 as signature_proto
import vega_sim.proto.vega.governance_pb2 as gov_proto

from nacl.signing import SigningKey
from nacl.encoding import HexEncoder, RawEncoder
import nacl.hash
import hashlib
import numpy as np

In [3]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

# Set up parties in the market/ Submit liquidity provision/ Control midprice
MM_WALLET = WalletConfig("mm", "pin")

# The party to send selling/buying MOs to hit LP orders
TRADER_WALLET = WalletConfig("Zl3pLs6Xk6SwIK7Jlp2x", "bJQDDVGAhKkj3PVCc7Rr")

# The party randomly post LOs at buy/sell side to simulate real Market situation
RANDOM_WALLET = WalletConfig("OJpVLvU5fgLJbhNPdESa", "GmJTt9Gk34BHDlovB7AJ")

# The party to terminate the market and send settlment price
TERMINATE_WALLET = WalletConfig("FJMKnwfZdd48C8NqvYrG", "bY3DxwtsCstMIIZdNpKs")

wallets = [MM_WALLET, TRADER_WALLET, RANDOM_WALLET, TERMINATE_WALLET]

In [4]:
vega = VegaServiceNull(run_with_console=True)
vega.start()

In [5]:
for wallet in wallets:
    vega.create_wallet(wallet.name, wallet.passphrase)

In [6]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=100000,
)

vega.forward("10s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=1e10,
)

In [7]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [8]:

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=1e5
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=1e2
)
vega.mint(
    RANDOM_WALLET.name,
    tdai_id,
    amount=1e5
)

In [9]:
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
    )

In [10]:
market_id = vega.all_markets()[0].id

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [11]:
market_id

'aca7307ba233e07c53719f38ba6818b7ae45c274852f1a6d0b57ff97e699ad14'

In [12]:
login_token = vega.login_tokens[MM_WALLET.name]
data_client = vega.trading_data_client()
pub_key = vega.pub_keys[MM_WALLET.name]

AttributeError: 'VegaServiceNull' object has no attribute 'login_tokens'

In [22]:
%%snakeviz
vega.submit_order(trading_wallet=MM_WALLET.name, market_id=market_id, 
    order_type='TYPE_LIMIT',
    time_in_force='TIME_IN_FORCE_GTC',
    side='SIDE_SELL',
    volume=10,
    price=1,)

 
*** Profile stats marshalled to file '/var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/tmp3cb2h__z'.
Embedding SnakeViz in this document...


In [509]:
headers = {"Authorization": f"Bearer {login_token}"}

order_ref = f"{pub_key}-{uuid.uuid4()}"

order_data = vega_protos.commands.v1.commands.OrderSubmission(
    market_id=market_id,
    # price is an integer. For example 123456 is a price of 1.23456,
    # assuming 5 decimal places.
    price="15",
    side=vega_protos.vega.Side.SIDE_SELL,
    size=10,
    time_in_force=vega_protos.vega.Order.TimeInForce.TIME_IN_FORCE_GTC,
    type=vega_protos.vega.Order.Type.TYPE_LIMIT,
    reference=order_ref,
)

In [490]:
submission = {
        "orderSubmission": MessageToDict(order_data),
        "pubKey": pub_key,
        "propagate": True,
    }

In [491]:
url = f"{vega.wallet_url()}/api/v1/command/sync"

In [492]:
enc = str(submission).encode()

In [493]:
# %%timeit
response = requests.post(url, headers=headers, json=submission)
response.json()

{'txHash': '506135537735514E617A6D5A77666D6F37534E497752774E5944726E7151487957306D7A4168464C6C52575A556E37596C494166724E46304E683469335A4736',
 'receivedAt': '2022-05-26T12:50:50.8906+01:00',
 'sentAt': '2022-05-26T12:50:50.898601+01:00',
 'txId': 'Nj2CvyrDTZDh2095GATQ',
 'tx': {'input_data': 'CNq8n6/x8I6LiwEQ8wXKPrUBCkA4MTQ4ZjkwZjRiOTk2ZWYwYWQzZjhlN2VmNzhjMzRkMzcxOWU3YmMzODk2ZGNiMWMxZjg5YjdlN2JiMGJiMzhlEgIxNRgKIAIoATgBQmU4MGVmOGQ1OTZhOGFmZjM1NjM1ZTM1NDBhZDhmNWI1ZTgyYTk3Zjg5ZTQ4OTMwMzg3ZDUwNjA5OGU1NWVlM2MxLWNkZWVkMDgzLWFlOWMtNDcyMi1hOTliLTY1YzgxOWFkYjE3MA==',
  'signature': {'value': '29be96198d4bdf7feeb1b83a3db5a3fccc76b035383cc68278a4397106e07d4d9935557b28591573f7bb80138c8a38c069e6bd0d5dbf0e4d5e509d3c5dd35c09',
   'algo': 'vega/ed25519',
   'version': 1},
  'From': {'PubKey': '80ef8d596a8aff35635e3540ad8f5b5e82a97f89e48930387d506098e55ee3c1'},
  'version': 2,
  'pow': {'tid': '48E6C3A2CB5CBEB2D2CD205B797EB6206394C1B0C4B6A6F3793D82F544B8A63C'}}}

In [392]:
sk = SigningKey.generate()

In [393]:
pub_key = sk.verify_key.encode(encoder=HexEncoder).decode()

In [514]:
faucet.mint(sk.verify_key.encode(encoder=HexEncoder).decode(), tdai_id, 10000, vega.faucet_url())
faucet.mint(sk.verify_key.encode(encoder=HexEncoder).decode(), "VOTE", 10000, vega.faucet_url())

In [395]:
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height

In [396]:
%%timeit
sk.sign(enc, encoder=HexEncoder)

80.7 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [397]:

def count_zeros(b):
    ret = 0
    for x in b:
        if x == 0:
            ret += 8
        else:
            if x % 128 != 0:
                break
            elif x % 64 != 0:
                ret += 1
                break
            elif x % 32 != 0:
                ret += 2
                break
            elif x % 16 != 0:
                ret += 3
                break
            elif x % 8 != 0:
                ret += 4
                break
            elif x % 4 != 0:
                ret += 5
                break
            elif x % 2 != 0:
                ret += 6
                break
            elif x % 1 != 0:
                ret += 7
                break
            break
    return ret

In [398]:
nonces = [(nonce).to_bytes(2, byteorder='big') for nonce in range(0, 10000)]
def PoW(signed_data, tx_id):
    valid_hash = None
    for nonce in nonces:
        new_key = b"Vega_SPAM_PoW" + signed_data + tx_id + nonce
        hashed = hashlib.sha256()
        hashed.update(new_key)
        hashed_dig = hashed.digest()
        if count_zeros(hashed_dig) >= 1:
            valid_hash = hashed_dig
            break
    return nonce, valid_hash

In [269]:
%%timeit
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height
signed = sk.sign(enc, encoder=HexEncoder)
tx_id = random.getrandbits(10*8).to_bytes(10, byteorder='big')
PoW(signed, tx_id)

529 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [343]:
block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height

In [344]:
block_height

760

In [523]:
order_data = vega_protos.commands.v1.commands.OrderSubmission(
    market_id=market_id,
    # price is an integer. For example 123456 is a price of 1.23456,
    # assuming 5 decimal places.
    price="8",
    side=vega_protos.vega.Side.SIDE_BUY,
    size=10,
    time_in_force=vega_protos.vega.Order.TimeInForce.TIME_IN_FORCE_GTC,
    type=vega_protos.vega.Order.Type.TYPE_LIMIT,
)

In [400]:
signed

b'09a27aedcb13db9db390cc0a6d90e3d9be81b79d9dd390c5878d4bfcce2c6d7461e687047b76d513f1f656b23703c25c03589d5c5ad3db7e04578f06ba3abf047b276f726465725375626d697373696f6e273a207b276d61726b65744964273a202765383732623932306236343562643266333034373436336366313838373832306364333631366461383566623165626539623732653634336631376137346534272c20277072696365273a20273135272c202773697a65273a20273130272c202773696465273a2027534944455f53454c4c272c202774696d65496e466f726365273a202754494d455f494e5f464f5243455f475443272c202774797065273a2027545950455f4c494d4954272c20277265666572656e6365273a2027313966333938373939333631396436356232373631616238616562663861363534336538633837393633653237643461633635383232333861376634613966622d35326233653966302d346233392d343830612d626664302d356162666534663065383964277d2c20277075624b6579273a202731396633393837393933363139643635623237363161623861656266386136353433653863383739363365323764346163363538323233386137663461396662272c202770726f706167617465273a20547275657d'

In [475]:
class sig_gen:
    
    def __init__(self):
        self.sigs = [os.urandom(6).hex() for _ in range(10000)]
        self.nonces = np.random.randint(0, 100000, size=100000)
        self.idx = 0
        self.nonce_idx = 0
        
    def get(self):
        self.idx += 1
        return self.sigs[self.idx]
    
    def get_nonce(self):
        self.nonce_idx += 1
        return self.nonces[self.nonce_idx]
    
sg = sig_gen()

In [457]:
%%timeit
os.urandom(6).hex()

359 ns ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [524]:
# %%timeit

input_data = transaction_proto.InputData(
    nonce = sg.get_nonce(),
    block_height = block_height,
    order_submission = order_data
)
serialised = input_data.SerializeToString()

trans = transaction_proto.Transaction(
        input_data=serialised,
        signature=signature_proto.Signature(
            value=sg.get(), 
            algo="vega/ed25519",
            version=1,
        ),
        pub_key=pub_key,
        version=2,
        
        pow=transaction_proto.ProofOfWork(
            tid=None,
            nonce=0, #int.from_bytes(nonce, byteorder='big'),
            hash_function="sha3_24_rounds"
        )
    )
request = core_proto.SubmitTransactionRequest(
    tx=trans,
    type=core_proto.SubmitTransactionRequest.Type.TYPE_ASYNC
)

vega.core_client().SubmitTransaction(request)

success: true
tx_hash: "737474394873736E50464E32506757724C594378766C486A4762765A7166786A6F644E575444367559424B3532566E65754F47653546346964356370647A4542"

In [525]:
vega.forward('10s')

In [479]:
vega.stop()

/var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/tmph2la5n74


In [127]:
%%timeit
vega.core_client().SubmitTransaction(request)

360 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [463]:
vega.console_port

59531

In [237]:
from vega_sim.proto.data_node.api.v1.trading_data_pb2 import GetVegaTimeRequest
blockchain_time = vega.trading_data_client().GetVegaTime(GetVegaTimeRequest()).timestamp

In [510]:
vega.core_state_client()

In [521]:
import vega_sim.proto.vega.api.v1.corestate_pb2 as corestate
markets = vega.core_state_client().ListMarketsData(corestate.ListMarketsDataRequest())

In [522]:
markets

markets_data {
  mark_price: "0"
  best_bid_price: "10"
  best_bid_volume: 10
  best_offer_price: "15"
  best_offer_volume: 10
  best_static_bid_price: "10"
  best_static_bid_volume: 10
  best_static_offer_price: "15"
  best_static_offer_volume: 10
  mid_price: "12"
  static_mid_price: "12"
  market: "2189efa1f5b48e97d17c164eff2ebf00b3aa14044ddf4740dd487900d8b7359a"
  timestamp: 1638185807259066000
  auction_end: 1638185788259066000
  auction_start: 1638185429259066000
  indicative_price: "0"
  market_trading_mode: TRADING_MODE_OPENING_AUCTION
  trigger: AUCTION_TRIGGER_OPENING
  target_stake: "0"
  supplied_stake: "10000"
  market_value_proxy: "10000"
  liquidity_provider_fee_share {
    party: "f28d251b2ca9198b65554f136b294da2282e3677b462d60b623de141da2d82e8"
    equity_like_share: "0"
    average_entry_valuation: "10000"
  }
}

In [273]:
propo = commands_proto.ProposalSubmission(
    reference="awdawfanaw",
    rationale=gov_proto.ProposalRationale(description="A test"),
    terms = gov_proto.ProposalTerms(
            closing_timestamp= blockchain_time + 400,
            enactment_timestamp= blockchain_time + 1000,
            validation_timestamp= blockchain_time + 100,
            update_network_parameter=gov_proto.UpdateNetworkParameter(
                changes = vega_protos.vega.NetworkParameter(
                    key = "market.fee.factors.makerFee",
                    value = "0.02",
                )
            )
    )
)

In [453]:
%%timeit
signed = sk.sign(enc, encoder=HexEncoder)
# block_height = vega.core_client().LastBlockHeight(core_proto.LastBlockHeightRequest()).height

hash_tx = hashlib.sha256()
hash_tx.update(os.urandom(10))
tx_byte = hash_tx.digest()
tx_hex = tx_byte.hex().upper()

nonce, proof = PoW(signed, tx_byte)

input_data = transaction_proto.InputData(
    nonce = random.randint(0, 1000000),
    block_height = block_height,
    proposal_submission = propo
)
serialised = input_data.SerializeToString()

trans = transaction_proto.Transaction(
        input_data=serialised,
        signature=signature_proto.Signature(
            value=signed,
            algo="vega/ed25519",
            version=1,
        ),
        pub_key=pub_key,
        version=2,
        pow=transaction_proto.ProofOfWork(
            tid=str(tx_hex),
            nonce=int.from_bytes(nonce, byteorder='big'),
            hash_function="sha3_24_rounds"
        )
    )
request = core_proto.SubmitTransactionRequest(
    tx=trans,
    type=core_proto.SubmitTransactionRequest.Type.TYPE_COMMIT
)

res = vega.core_client().SubmitTransaction(request)

1.24 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [283]:
transaction_proto.InputData().ParseFromString(serialised)

103

In [530]:
import inflection

In [533]:
inflection.camelize('orderSubmission', uppercase_first_letter=False)

'orderSubmission'